# calculate rotation and transposition matrix

In [15]:
data_folder = r'D:\Shiwei\20210423-P_Forebrain_smRNA'
before_position_file = os.path.join(data_folder, '10x_positions_before.txt')
after_position_file = os.path.join(data_folder, '10x_positions_after.txt')


In [16]:
import numpy as np
import os, sys
# 1. alignment for manually picked points
def align_manual_points(pos_file_before, pos_file_after,
                        save=True, save_folder=None, save_filename='', verbose=True):
    """Function to align two manually picked position files, 
    they should follow exactly the same order and of same length.
    Inputs:
        pos_file_before: full filename for positions file before translation
        pos_file_after: full filename for positions file after translation
        save: whether save rotation and translation info, bool (default: True)
        save_folder: where to save rotation and translation info, None or string (default: same folder as pos_file_before)
        save_filename: filename specified to save rotation and translation points
        verbose: say something! bool (default: True)
    Outputs:
        R: rotation for positions, 2x2 array
        T: traslation of positions, array of 2
    Here's example for how to translate points
        translated_ps_before = np.dot(ps_before, R) + t
    """
    # load position_before
    if os.path.isfile(pos_file_before):
        ps_before = np.loadtxt(pos_file_before, delimiter=',')

    # load position_after
    if os.path.isfile(pos_file_after):
        ps_after = np.loadtxt(pos_file_after, delimiter=',')

    # do SVD decomposition to get best fit for rigid-translation
    c_before = np.mean(ps_before, axis=0)
    c_after = np.mean(ps_after, axis=0)
    H = np.dot((ps_before - c_before).T, (ps_after - c_after))
    U, _, V = np.linalg.svd(H)  # do SVD
    # calcluate rotation
    R = np.dot(V, U.T).T
    if np.linalg.det(R) < 0:
        R[:, -1] = -1 * R[:, -1]
    # calculate translation
    t = - np.dot(c_before, R) + c_after
    # here's example for how to translate points
    # translated_ps_before = np.dot(ps_before, R) + t
    # save
    if save:
        if save_folder is None:
            save_folder = os.path.dirname(pos_file_before)
        if not os.path.exists(save_folder):
            os.makedirs(save_folder)
        if len(save_filename) > 0:
            save_filename += '_'
        rotation_name = os.path.join(save_folder, save_filename+'rotation')
        translation_name = os.path.join(
            save_folder, save_filename+'translation')
        np.save(rotation_name, R)
        np.save(translation_name, t)

    return R, t

In [17]:
R, T = align_manual_points(before_position_file, after_position_file, save=False)


In [19]:
R, T

(array([[ 0.99997965, -0.00637995],
        [ 0.00637995,  0.99997965]]), array([-378.32416804,  729.82026289]))

# transpose 60x positions

In [20]:
old_positions = np.loadtxt(os.path.join(data_folder, 'positions_all.txt'), delimiter=',')

In [21]:
new_positions = np.dot(old_positions, R) + T
print(new_positions)

[[-2917.88498205   947.82696446]
 [-2916.47373766  1169.02246259]
 [-2915.06249327  1390.21796071]
 [-2913.65188688  1611.31346086]
 [-2912.24064249  1832.50895899]
 [-2910.8293981   2053.70445711]
 [-2689.63389998  2052.29321272]
 [-2691.04514437  1831.0977146 ]
 [-2692.45638876  1609.90221648]
 [-2693.86699515  1388.80671632]
 [-2695.27823954  1167.6112182 ]
 [-2696.68948393   946.41572008]
 [-2475.49398581   945.00447569]
 [-2474.08274142  1166.19997381]
 [-2472.67149703  1387.39547193]
 [-2471.26089064  1608.49097209]
 [-2469.84964625  1829.68647021]
 [-2468.43840186  2050.88196833]
 [-2247.3429017   2049.47136194]
 [-2248.75414609  1828.27586382]
 [-2250.16539048  1607.08036569]
 [-2251.57599687  1385.98486554]
 [-2252.98724126  1164.78936742]
 [-2254.39848565   943.59386929]
 [-2033.20298753   942.18262491]
 [-2031.79174314  1163.37812303]
 [-2030.38049875  1384.57362115]
 [-2028.96989236  1605.66912131]
 [-2027.55864797  1826.86461943]
 [-2026.14740358  2048.06011755]
 [-1804.95

In [22]:
save_filename = os.path.join(data_folder, 'translated_positions_all.txt')
print(save_filename)
#np.savetxt(save_filename, new_positions, fmt='%.2f', delimiter=',')

D:\Shiwei\20210423-P_Forebrain_smRNA\translated_positions_all.txt


# further adjust manually

In [8]:
manual_shift = np.array([-28.1,  -8.7])
adjusted_new_positions = new_positions + manual_shift

In [9]:
adj_save_filename = os.path.join(data_folder, 'adjusted_translated_positions_all.txt')
print(adj_save_filename)
np.savetxt(adj_save_filename, adjusted_new_positions, fmt='%.2f', delimiter=',')

\\storm6-pc\STORM6-FLASHDrive\Pu\20201127-NOAcr_CTP-08_E14_brain_no_clearing\adjusted_translated_positions_all.txt
